In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries

In [10]:
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

## Load Data

In [11]:
# Load environment variables
load_dotenv()
jwt_token = os.getenv('JWT_TOKEN')

# Define headers for API requests
headers = {
    'Authorization': f'Bearer {jwt_token}'
}

# API URLs
api_urls = {
    'interactions': 'http://161.97.109.65:3000/api/interactions/download',
    'users': 'http://161.97.109.65:3000/api/users/download',
    'products': 'http://161.97.109.65:3000/api/products/download'
}

# Fetch data from APIs
def fetch_data(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    else:
        print(f'Failed to fetch data: {response.status_code}')
        return pd.DataFrame()

users = fetch_data(api_urls['users'], headers)
products = fetch_data(api_urls['products'], headers)
interactions = fetch_data(api_urls['interactions'], headers)


Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403


## Data Preprocessing

In [13]:
# Encode user_id and product_id
user_ids = interactions['user_id'].unique().tolist()
product_ids = interactions['product_id'].unique().tolist()

user2user_encoded = {x: i for i, x in enumerate(user_ids)}
product2product_encoded = {x: i for i, x in enumerate(product_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
productencoded2product = {i: x for i, x in enumerate(product_ids)}

interactions['user'] = interactions['user_id'].map(user2user_encoded)
interactions['product'] = interactions['product_id'].map(product2product_encoded)

# Split the data
train, test = train_test_split(interactions, test_size=0.2, random_state=42)

# Convert data into required format
x_train = [train['user'].values, train['product'].values]
y_train = train['interaction_value'].values
x_test = [test['user'].values, test['product'].values]
y_test = test['interaction_value'].values

KeyError: 'user_id'

## Define Model

In [ ]:
# Define the model
user_input = Input(shape=(1,))
user_embedding = Embedding(len(user_ids), 50, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

product_input = Input(shape=(1,))
product_embedding = Embedding(len(product_ids), 50, input_length=1)(product_input)
product_vec = Flatten()(product_embedding)

dot_product = Dot(axes=1)([user_vec, product_vec])

model = Model([user_input, product_input], dot_product)
model.compile(optimizer='adam', loss='mean_squared_error')

## Train Model

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=20, verbose=1, validation_split=0.2)

## Try Model

In [ ]:
# Function to get recommendations for a specific user
def recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products, top_n=30):
    user_encoded = user2user_encoded[user_id]
    product_ids = list(product2product_encoded.values())
    user_product_array = np.hstack(
        ([[user_encoded]] * len(product_ids), np.array(product_ids).reshape(-1, 1))
    )

    predictions = model.predict([user_product_array[:, 0], user_product_array[:, 1]])
    predictions = predictions.flatten()

    top_indices = predictions.argsort()[-top_n:][::-1]
    recommended_product_ids = [productencoded2product[x] for x in top_indices]

    recommended_products = products[products['product_id'].isin(recommended_product_ids)]
    return recommended_products

In [ ]:
# Try the model
user_id = 23
recommended_products = recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products)
print(f"Recommended products for user {user_id}:")
print(recommended_products)